In [2]:
import networkx  as nx
import matplotlib.pyplot as plt
import pandas as pd
import powerlaw
import numpy as np


In [7]:
df_edge = pd.read_csv("../data/edges.csv").astype("str")

,package,requirement
0,17,3544
1,17,11976
2,17,12143
3,17,18112
4,17,19172
...,...,...
73768,26392,20683
73769,26392,23924
73770,26392,23965
73771,26393,14147


In [ ]:
G = nx.from_pandas_edgelist(df_edge,"package", "requirement",create_using=nx.Graph)
degree_sequence = sorted((d for n, d in G.degree()), reverse=True)

In [ ]:
node_with_highest_degree = max(G.nodes(), key=G.degree)
highest_degree = G.degree(node_with_highest_degree)
print("Node with highest degree:", node_with_highest_degree)
print("Degree:", highest_degree)

In [ ]:
def distribution(degree):
    hist, bins_edge = np.histogram(degree, bins=np.arange(0.5,10**4+1.5,1), density=True)
    
    P = hist*np.diff(bins_edge)             # distribution = density*deltaK
    K = bins_edge[:-1]+bins_edge[:1]
    index_remove = []                       # load index with distribution zero
    
    for idk,elements in enumerate(P):
        if(elements==0):
            index_remove.append(idk) 
    # Removing elements in k_mean and distribution with distribution = 0 (empty box)
    p_real = np.delete(P,index_remove)      
    k_real = np.delete(K,index_remove)
    return k_real,p_real

In [ ]:
k, pk = distribution(degree_sequence)

In [ ]:
data = degree_sequence
fit = powerlaw.Fit(data, discrete=True)
####
figCCDF = fit.plot_pdf(color='b', linewidth=2,label="PDF")
fit.power_law.plot_pdf(color='b', linestyle='--', ax=figCCDF,label="Regression")
fit.plot_ccdf(color='r', linewidth=2, ax=figCCDF,label="CDF")
fit.power_law.plot_ccdf(color='r', linestyle='--', ax=figCCDF,label="Regression")

handles, labels = figCCDF.get_legend_handles_labels()
leg = figCCDF.legend(handles, labels, loc=3)
leg.draw_frame(False)
####
figCCDF.set_ylabel(u"Distributions")
figCCDF.set_xlabel(r"Degree")
filename = '../results/Distributions'
plt.savefig(filename+'.pdf', bbox_inches='tight')
#savefig(figname+'.eps', bbox_inches='tight')


In [ ]:
fit.alpha
#fit.xmin